In [75]:
from tensorflow.keras.models import load_model
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import os
from math import *
from discretize import TensorMesh

from discretize import TreeMesh
from discretize.utils import mkvc, refine_tree_xyz
from discretize.utils import mkvc, refine_tree_xyz, active_from_xyz

from SimPEG.utils import plot2Ddata, surface2ind_topo
import SimPEG.utils as Utils
from SimPEG import maps
import SimPEG.electromagnetics.frequency_domain as fdem
import SimPEG.electromagnetics.time_domain as tdem
from discretize import TensorMesh
from math import *
import time



In [76]:
#Input_conf.shape,X_train.shape,Y_train.shape
#plt.plot(X_train[K,:,0,1])

In [49]:
# Load data
X_train=np.load('X_train.npy')
Y_train=np.load('Y_train.npy')
Input_conf=np.load('Input_conf.npy')



def addnoise(K1,val0,por):
    val=np.copy(val0)
    valk=[]
    for i in range(len(val)-1):
        valk.append(-val[i])
        if(val[i]==0):
            break
    valk=np.power(10,valk)
    k=K1*1000
    np.random.seed(k)
    random_numbers = np.random.normal(loc=0, scale=1, size=len(valk))
    #print(valk)
    random_numbers_clipped = np.clip(random_numbers, -1, 1)
    valk=valk+valk*por*random_numbers_clipped
    val[:len(valk)]=np.abs(np.log10(valk))
    return val


# Add Gauss noise for 40,000 samples
X_train_add15=np.copy(X_train[:20000,:,:,:])
X_train_add20=np.copy(X_train[20000:40000,:,:,:])
Y_train_add15=np.copy(Y_train[:20000,:])
Y_train_add20=np.copy(Y_train[20000:40000,:])

for i in range(20000):
    valtem=addnoise(i,X_train_add15[i,:,0,1],0.15)
    X_train_add15[i,:,0,1]=valtem
    
for i in range(20000):
    valtem=addnoise(i+20000,X_train_add20[i,:,0,1],0.2)
    X_train_add20[i,:,0,1]=valtem
    

    
X_train = np.concatenate((X_train, X_train_add15),axis=0)
X_train = np.concatenate((X_train, X_train_add20),axis=0)

Y_train = np.concatenate((Y_train, Y_train_add15),axis=0)
Y_train = np.concatenate((Y_train, Y_train_add20),axis=0)

# K=10
# valtem=addnoise(0,X_train[K,:,0,1],0.2)

# plt.plot(X_train[K,:20,0,0],X_train[K,:20,0,1])
# plt.plot(X_train[K,:20,0,0],valtem[:20])

In [74]:
# plt.plot(X_train_add20[1500,:,0,1])
# plt.plot(X_train[21500,:,0,1])

In [59]:
# sample balancing
np.random.seed(122) #122
index_data=np.arange(0,len(X_train),1)
np.random.shuffle(index_data)

X_train_tf=X_train[index_data]
Y_train_tf=Y_train[index_data]
X_train_tf.shape,Y_train.shape

((137000, 32, 1, 2), (137000, 100))

In [62]:
#Create ResNet framework

import tensorflow as tf

# Define the residual block
def residual_block(x, filters):
    shortcut = x
    
    # First convolutional layer
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    
    # Second convolutional layer
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    
    # Residual connection
    x = tf.keras.layers.add([shortcut, x])
    
    # Apply ReLU activation
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

# Build the residual convolutional neural network model
def build_model():
    inputs = tf.keras.Input(shape=(32, 1, 2))
    
    # Initial convolutional layer
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    
    # Residual blocks
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)
    
    # Global average pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    outputs = tf.keras.layers.Dense(100, activation='sigmoid')(x)
    #outputs = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model = build_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 1, 2)]   0           []                               
                                                                                                  
 conv2d_15 (Conv2D)             (None, 32, 1, 32)    608         ['input_4[0][0]']                
                                                                                                  
 conv2d_16 (Conv2D)             (None, 32, 1, 32)    9248        ['conv2d_15[0][0]']              
                                                                                                  
 conv2d_17 (Conv2D)             (None, 32, 1, 32)    9248        ['conv2d_16[0][0]']              
                                                                                            

In [68]:
#Model training

csv_logger = tf.keras.callbacks.CSVLogger('CNN/training_20240213.log')
test_interval=int(0.1*len(X_train))

#model.fit(X_train, Y_train, epochs=10, batch_size=32)

history = model.fit(X_train_tf[:-test_interval], Y_train_tf[:-test_interval],batch_size=300,verbose=1,\
                    epochs=50, validation_data=(X_train_tf[-test_interval:], Y_train_tf[-test_interval:]),\
                    callbacks=[csv_logger])

Epoch 1/50
411/411 [==============================] - 8s 19ms/step - loss: 0.1007 - accuracy: 0.4564 - val_loss: 0.1009 - val_accuracy: 0.4564
Epoch 2/50
411/411 [==============================] - 8s 18ms/step - loss: 0.1011 - accuracy: 0.4523 - val_loss: 0.1007 - val_accuracy: 0.4569
Epoch 3/50
411/411 [==============================] - 8s 18ms/step - loss: 0.1015 - accuracy: 0.4491 - val_loss: 0.1014 - val_accuracy: 0.4364
Epoch 4/50
411/411 [==============================] - 8s 19ms/step - loss: 0.1012 - accuracy: 0.4517 - val_loss: 0.1040 - val_accuracy: 0.4332
Epoch 5/50
411/411 [==============================] - 8s 19ms/step - loss: 0.1009 - accuracy: 0.4539 - val_loss: 0.1017 - val_accuracy: 0.4554
Epoch 6/50
411/411 [==============================] - 8s 19ms/step - loss: 0.1006 - accuracy: 0.4543 - val_loss: 0.1014 - val_accuracy: 0.4524
Epoch 7/50
411/411 [==============================] - 8s 19ms/step - loss: 0.1013 - accuracy: 0.4512 - val_loss: 0.1046 - val_accuracy: 0.4436

In [73]:
# Save model and weight

model.save_weights('CNN/Invdep20240213_.ckpt')
model.save('CNN/Invdep20240213.h5')

In [70]:
prediction = model.predict(X_train)

4282/4282 [==============================] - 8s 2ms/step
